In [85]:
using LinearAlgebra
using PlotlyJS
using FFTW
using BenchmarkTools
using Kronecker

In [86]:
x = collect(LinRange(0, 127, 128)) + 1im * collect(LinRange(-64, 63, 128));

# Comparison between naive DFT $O(n^2)$, Radix 2-FFT $O(nlogn)$ and Julia's FFTW 

In [87]:
function dft_matrix(n)
    """ 
    Computes the Discrete Fourier Transform matrix of size n.
    Based on algorithm 1.16 from
    Van Loan, C. (1992). Computational frameworks for the fast Fourier transform.
    
    Input: 
    n: (integer)
    
    Returns:
    F: (n x n complex matrix)
    """
    
    # Base cases
    if n == 1
        return 1
    end
    
    F = ones(ComplexF64, n, n)
    F[1, 2] = 1
    
    for p = 1:n-1
        F[p + 1, 2] = exp(-2 * pi * 1im * p/n)
    end
    
    for q = 2:n-1
        F[:, q + 1] = F[:, q] .* F[:, 2]
    end

    return F
end

function naive_dft(x)
    """ 
    Computes the Discrete Fourier Transform of an input x.
    
    Input: 
    x: (vector)
    
    Returns:
    y: (vector)
    """
    
    return dft_matrix(length(x)) * x
end

function naive_idft(y)
    """ 
    Computes the inverse Discrete Fourier Transform of an input y.
    
    Input: 
    y: (vector)
    
    Returns:
    x: (vector)
    """
    
    return (dft_matrix(length(y))' * y) / length(y)
end
    

naive_idft (generic function with 1 method)

In [88]:
function radix2_fft(x)
    """ 
    Computes the Fast Fourier Transform of an input x, using a recursive
    radix2 procedure.
    
    Input: 
    x: (vector)
    
    Returns:
    y: (vector)
    """
    
    return radix2_fft_base(x, length(x), false)
end

function radix2_ifft(y)
    """ 
    Computes the inverse Fast Fourier Transform of an input x, using a recursive
    radix2 procedure.
    
    Input: 
    y: (vector)
    
    Returns:
    x: (vector)
    """
    
    return radix2_fft_base(y, length(y), true) / length(y)
end
    
function radix2_fft_base(x, n, inverse)
    """ 
    Base routine for recursive
    radix2 procedure for FFT computation.
    Based on 1.26 from
    Van Loan, C. (1992). Computational frameworks for the fast Fourier transform.
    
    Input: 
    x: (vector)
    n: (integer, size of fft)
    inverse: (bool, if doing fft or ifft)
    
    Returns:
    y: (vector)
    """
    
    pwr_sign = inverse ? 1 : -1
            
    if n == 1
        return x
    
    else
        m = n÷2
        ω = exp(pwr_sign * 2 * pi * 1im/n)
        Ω = [ω^p for p in 0:m-1]
        z_t = radix2_fft_base(x[1:2:n], m, inverse)
        z_b = Ω .* radix2_fft_base(x[2:2:n], m, inverse)
        y = [z_t + z_b;z_t - z_b]
        
        return y
    end
        
end
        

radix2_fft_base (generic function with 1 method)

In [89]:
@btime y1 = fft(x)

  45.000 μs (5 allocations: 2.42 KiB)


128-element Vector{ComplexF64}:
              8128.0 - 63.999999999999986im
 -2671.0709678133317 + 2543.0709678133317im
   -1366.74992799918 + 1238.74992799918im
  -931.6268315105551 + 803.6268315105551im
   -713.802904806967 + 585.802904806967im
  -582.8982914353221 + 454.89829143532194im
  -495.4529539465592 + 367.4529539465593im
 -432.84108832760364 + 304.8410883276038im
 -385.74972749605433 + 257.7497274960543im
 -349.00494236252234 + 221.00494236252234im
 -319.50232216128546 + 191.50232216128535im
 -295.26628360367533 + 167.26628360367545im
  -274.9797253720525 + 146.97972537205243im
                     ⋮
   146.9797253720525 - 274.9797253720525im
   167.2662836036755 - 295.26628360367533im
   191.5023221612854 - 319.5023221612855im
  221.00494236252212 - 349.00494236252223im
  257.74972749605433 - 385.74972749605433im
  304.84108832760387 - 432.84108832760387im
   367.4529539465593 - 495.45295394655926im
    454.898291435322 - 582.8982914353221im
    585.802904806967 - 713.80290

In [90]:
y1 = fft(x)

128-element Vector{ComplexF64}:
              8128.0 - 63.999999999999986im
 -2671.0709678133317 + 2543.0709678133317im
   -1366.74992799918 + 1238.74992799918im
  -931.6268315105551 + 803.6268315105551im
   -713.802904806967 + 585.802904806967im
  -582.8982914353221 + 454.89829143532194im
  -495.4529539465592 + 367.4529539465593im
 -432.84108832760364 + 304.8410883276038im
 -385.74972749605433 + 257.7497274960543im
 -349.00494236252234 + 221.00494236252234im
 -319.50232216128546 + 191.50232216128535im
 -295.26628360367533 + 167.26628360367545im
  -274.9797253720525 + 146.97972537205243im
                     ⋮
   146.9797253720525 - 274.9797253720525im
   167.2662836036755 - 295.26628360367533im
   191.5023221612854 - 319.5023221612855im
  221.00494236252212 - 349.00494236252223im
  257.74972749605433 - 385.74972749605433im
  304.84108832760387 - 432.84108832760387im
   367.4529539465593 - 495.45295394655926im
    454.898291435322 - 582.8982914353221im
    585.802904806967 - 713.80290

In [91]:
@btime y2 = radix2_fft(x)

  75.002 μs (889 allocations: 112.12 KiB)


128-element Vector{ComplexF64}:
              8128.0 - 63.999999999999986im
  -2671.070967813331 + 2543.0709678133317im
   -1366.74992799918 + 1238.74992799918im
  -931.6268315105556 + 803.6268315105553im
  -713.8029048069669 + 585.8029048069671im
  -582.8982914353219 + 454.89829143532194im
  -495.4529539465591 + 367.4529539465591im
  -432.8410883276034 + 304.8410883276035im
 -385.74972749605445 + 257.74972749605433im
 -349.00494236252246 + 221.00494236252229im
  -319.5023221612856 + 191.50232216128538im
  -295.2662836036756 + 167.26628360367545im
  -274.9797253720526 + 146.97972537205257im
                     ⋮
   146.9797253720528 - 274.9797253720527im
  167.26628360367573 - 295.2662836036755im
  191.50232216128575 - 319.5023221612856im
  221.00494236252257 - 349.004942362522im
   257.7497274960549 - 385.7497274960546im
   304.8410883276045 - 432.8410883276041im
   367.4529539465601 - 495.45295394655966im
   454.8982914353233 - 582.8982914353228im
   585.8029048069682 - 713.80290480

In [92]:
y2 = radix2_fft(x)

128-element Vector{ComplexF64}:
              8128.0 - 63.999999999999986im
  -2671.070967813331 + 2543.0709678133317im
   -1366.74992799918 + 1238.74992799918im
  -931.6268315105556 + 803.6268315105553im
  -713.8029048069669 + 585.8029048069671im
  -582.8982914353219 + 454.89829143532194im
  -495.4529539465591 + 367.4529539465591im
  -432.8410883276034 + 304.8410883276035im
 -385.74972749605445 + 257.74972749605433im
 -349.00494236252246 + 221.00494236252229im
  -319.5023221612856 + 191.50232216128538im
  -295.2662836036756 + 167.26628360367545im
  -274.9797253720526 + 146.97972537205257im
                     ⋮
   146.9797253720528 - 274.9797253720527im
  167.26628360367573 - 295.2662836036755im
  191.50232216128575 - 319.5023221612856im
  221.00494236252257 - 349.004942362522im
   257.7497274960549 - 385.7497274960546im
   304.8410883276045 - 432.8410883276041im
   367.4529539465601 - 495.45295394655966im
   454.8982914353233 - 582.8982914353228im
   585.8029048069682 - 713.80290480

In [93]:
@btime y3 = naive_dft(x)

  297.005 μs (381 allocations: 1.04 MiB)


128-element Vector{ComplexF64}:
              8128.0 - 64.0im
  -2671.070967813332 + 2543.0709678133385im
  -1366.749927999184 + 1238.7499279991898im
  -931.6268315105585 + 803.6268315105578im
  -713.8029048069666 + 585.8029048069641im
   -582.898291435322 + 454.8982914353203im
 -495.45295394655847 + 367.45295394655545im
  -432.8410883276047 + 304.84108832760495im
  -385.7497274960541 + 257.7497274960541im
  -349.0049423625228 + 221.00494236252212im
 -319.50232216128586 + 191.50232216128558im
   -295.266283603678 + 167.26628360367386im
  -274.9797253720527 + 146.97972537205106im
                     ⋮
   146.9797253720513 - 274.97972537205203im
   167.2662836036812 - 295.266283603675im
  191.50232216129712 - 319.5023221612805im
  221.00494236254127 - 349.00494236251626im
   257.7497274960833 - 385.74972749604564im
    304.841088327601 - 432.8410883276053im
   367.4529539465668 - 495.45295394655614im
   454.8982914353419 - 582.8982914353171im
   585.8029048070032 - 713.8029048069453im
 

In [94]:
y3 = naive_dft(x)

128-element Vector{ComplexF64}:
              8128.0 - 64.0im
  -2671.070967813332 + 2543.0709678133385im
  -1366.749927999184 + 1238.7499279991898im
  -931.6268315105585 + 803.6268315105578im
  -713.8029048069666 + 585.8029048069641im
   -582.898291435322 + 454.8982914353203im
 -495.45295394655847 + 367.45295394655545im
  -432.8410883276047 + 304.84108832760495im
  -385.7497274960541 + 257.7497274960541im
  -349.0049423625228 + 221.00494236252212im
 -319.50232216128586 + 191.50232216128558im
   -295.266283603678 + 167.26628360367386im
  -274.9797253720527 + 146.97972537205106im
                     ⋮
   146.9797253720513 - 274.97972537205203im
   167.2662836036812 - 295.266283603675im
  191.50232216129712 - 319.5023221612805im
  221.00494236254127 - 349.00494236251626im
   257.7497274960833 - 385.74972749604564im
    304.841088327601 - 432.8410883276053im
   367.4529539465668 - 495.45295394655614im
   454.8982914353419 - 582.8982914353171im
   585.8029048070032 - 713.8029048069453im
 

In [95]:
norm(ifft(y1) - x, Inf)

2.842170943040401e-14

In [96]:
norm(radix2_ifft(y2) - x, Inf)

4.14314051144892e-13

In [97]:
norm(naive_idft(y3) - x, Inf)

2.3017773928165964e-12

# Finding Cooley - Tukey factorization permutation matrix

### Naive method (using Kroenecker products)

In [160]:
function cooley_tukey_perm_naive(n)
    """ 
    Finds the Cooley Tukey algorithm permutation matrix using Kroenecker products 
    (naive approach)
    
    Input: 
    n: (integer, size of fft)
    
    Returns:
    P: (permutation matrix)
    """ 
    
    P = I + zeros(n, n)
    t = Int(log2(n))
    
    for q = 1:t-1
        Itq = I + zeros(2^(t-q), 2^(t-q))
        Iq = I + zeros(2^q, 2^q)
        perm = vcat(1:2:2^q, 2:2:2^q);
        Πq = Iq[:, perm]
                
        Rq = kronecker(Itq, Πq) 
        P = Rq * P
        
    end
    
    # Last R matrix
    perm = vcat(1:2:n, 2:2:n)
    It = I + zeros(n, n)
    Πt = It[:, perm]
    Rt = Πt
    
    P = Rt * P
    
    return P
end


cooley_tukey_perm_naive (generic function with 1 method)

In [162]:
cooley_tukey_perm_naive(8)

8×8 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

In [163]:
function bit_reverse(i, bits)
    r = 0
    for _ in 1:bits
        r = (r << 1) | (i & 1)  # Shift left and add least significant bit
        i >>= 1                 # Shift input right
    end
    return r
end

bit_reverse (generic function with 1 method)

#### Small example on how bit_reverse function operates

| Step | i (Binary) | Extract i & 1 | r (Binary) Before | r (Binary) After |
|------|------------|---------------|-------------------|------------------|
| 1    | 110        | 0             | 000               | 000              |
| 2    | 011        | 1             | 000               | 001              |
| 3    | 001        | 1             | 001               | 011              |

In [203]:
function cooley_tukey_perm(n, return_matrix = false)
    """ 
    Finds the Cooley Tukey algorithm permutation matrix using bitwise operations.
    
    Input: 
    n: (integer, size of fft)
    
    Returns:
    P: (permutation matrix)
    """ 
    

    t = Int(log2(n))
    perm = bit_reverse.(collect(0:n-1), t) .+1
    
    if !return_matrix
        return perm
    end
    
    P = I + zeros(n, n)
    
    return P[:, perm] 
end


cooley_tukey_perm (generic function with 2 methods)

In [197]:
norm(cooley_tukey_perm(16, true) - cooley_tukey_perm_naive(16), 1)

0.0

In [214]:
function cooley_tukey_fft_base(x, inverse = false)
    """ 
    Base function to computes the Cooley Tukey FFT/IFFT. Based on algorithm 1.9.3 from
    Van Loan, C. (1992). Computational frameworks for the fast Fourier transform.

    
    Input: 
    x: (vector)
    
    Returns:
    y: (vector)
    """ 
    n = length(x)
    p = cooley_tukey_perm(n)
    t = Int(log2(n))
    # Applying permutation
    y = x[p]    
    pwr_sign = inverse ? 1 : -1
    for q = 1:t
        L = 2^q
        r = n ÷ L
        L_star = L ÷ 2
        for j = 0:L_star - 1 
            ω = cos(2 * pi * j / L) + pwr_sign * 1im * sin(2 * pi * j/L)
            for k = 0:r-1
                τ = ω * y[1 + (k * L + j + L_star)]
                y[1 + (k * L + j + L_star)] = y[1 + (k * L + j)] - τ
                y[1 + (k * L + j)] = y[1 + (k * L + j)] + τ
            end
        end
    end

    return inverse ? y/length(y) : y
end


function cooley_tukey_fft(x)
    return cooley_tukey_fft_base(x)
end

function cooley_tukey_ifft(y)
    return cooley_tukey_fft_base(y, true)
end



cooley_tukey_ifft (generic function with 1 method)

In [216]:
cooley_tukey_ifft(cooley_tukey_fft(x))

128-element Vector{ComplexF64}:
 3.552713678800501e-15 - 63.999999999999986im
     1.000000000000007 - 62.99999999999999im
    2.0000000000000178 - 61.99999999999999im
     3.000000000000007 - 60.999999999999986im
     4.000000000000021 - 59.999999999999986im
     5.000000000000011 - 58.999999999999986im
     6.000000000000018 - 57.999999999999986im
     7.000000000000018 - 56.99999999999999im
     8.000000000000007 - 55.999999999999986im
                   9.0 - 54.999999999999986im
     10.00000000000001 - 53.999999999999986im
                  11.0 - 52.999999999999986im
    12.000000000000014 - 51.999999999999986im
                       ⋮
                 116.0 + 52.000000000000014im
                 117.0 + 53.00000000000001im
    118.00000000000001 + 54.0im
    119.00000000000001 + 55.00000000000001im
                 120.0 + 56.00000000000001im
                 121.0 + 57.000000000000014im
                 122.0 + 58.00000000000003im
                 123.0 + 58.999999999999986i